# Imports

In [1]:
import oauth2
import json
import time
import datetime
import os
import sys

from kafka import KafkaProducer

# Configuration

In [2]:

start_date = "2020-01-01" 
end_date = "2020-03-03"
consumerKey="XXXXXXXXXXXXXXXXXX"
consumerSecret="XXXXXXXXXXXXXXXXXX"
accessToken="XXXXXXXXXXXXXXXXXX"
accessTokenSecret="XXXXXXXXXXXXXXXXXX"
keyword= "superbowl"
lang="en"
kafka_url="localhost:9092"
topic_name= "raw-twitter"


req_count = 0
min_faves=60000
change=10000 
interval = 500 

# Authentication Method

In [3]:
def oauth_req(url, http_method="GET", post_body=b"", http_headers=None):
    global req_count,consumerKey,consumerSecret,accessToken,accessTokenSecret
    req_count += 1
    consumer = oauth2.Consumer(key=consumerKey, secret=consumerSecret)
    token = oauth2.Token(key=accessToken, secret=accessTokenSecret)
    client = oauth2.Client(consumer, token)
    resp, content = client.request( url, method=http_method, body=post_body , headers=http_headers )
    return content

# Get Tweets URL Construction

In [4]:
def get_tweets(min_faves):
    global keyword, start_date, end_date, lang
    request = 'https://api.twitter.com/1.1/search/tweets.json?' + '&q=' + keyword + '&lang=' + lang + '%20since%3A' + start_date + '%20until%3A' + end_date + '%20min_faves%3A' + str(min_faves) +'&result_type=mixed&count=100'
    return oauth_req( request)

# Publish Message to Kakfa

In [5]:
def connect_kafka_producer():
    producer = None
    try:
        producer = KafkaProducer(bootstrap_servers=[kafka_url], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return producer
    
def publish_message(producer_instance, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

# KeepAlive Process

In [7]:
kafka_producer = connect_kafka_producer()


while(1): 
    data = {}
    new_data_raw = json.loads(get_tweets(min_faves))
    try:
        for i in new_data_raw['statuses']:
            data[i['id']] = i
        counter = len(new_data_raw['statuses'])
    except Exception as ex:
        print("Error at request : " + str(req_count))
        print(str(ex))
    publish_message(kafka_producer, 'raw', json.dumps(data))
    print("Sleeping")
    time.sleep(10)
    print("Waking Up")
    print("At request: " + str(req_count) + "  Total Tweets Collected: " + str(len(data)) + " with Min Faves: " + str(min_faves) )
    if counter==100 and min_faves>10000:
        if (change>1000):
            change /= 2
            min_faves += change
        else:
            min_faves -= change
        
    elif min_faves>10000:
        min_faves -= change
    
    else:
        min_faves -= interval
        if(min_faves < 0):
            break


Message published successfully.
Sleeping
Waking Up
At request: 2  Total Tweets Collected: 0 with Min Faves: 60000
Message published successfully.
Sleeping
Waking Up
At request: 3  Total Tweets Collected: 0 with Min Faves: 50000
Message published successfully.
Sleeping
Waking Up
At request: 4  Total Tweets Collected: 1 with Min Faves: 40000
Message published successfully.
Sleeping
Waking Up
At request: 5  Total Tweets Collected: 1 with Min Faves: 30000
Message published successfully.
Sleeping
Waking Up
At request: 6  Total Tweets Collected: 1 with Min Faves: 20000
Message published successfully.
Sleeping
Waking Up
At request: 7  Total Tweets Collected: 4 with Min Faves: 10000
Message published successfully.
Sleeping
Waking Up
At request: 8  Total Tweets Collected: 4 with Min Faves: 9500
Message published successfully.
Sleeping
Waking Up
At request: 9  Total Tweets Collected: 4 with Min Faves: 9000
Message published successfully.
Sleeping
Waking Up
At request: 10  Total Tweets Collected: